# Agents for Impact: Medical Diet Navigator Challenge

## Step 1

This notebook is used to import the USDA Food Data Central Database into BigQuery. It does the following:

1. Downloads and unzips the USDA data file (CSV) located here: https://fdc.nal.usda.gov/fdc-datasets/FoodData_Central_foundation_food_csv_2025-04-24.zip

2. Creates a dataset and loads the data from the CSV files into BigQuery tables.

The data is used by the ADK BigQuery Agent (configured in Step 2) to retrieve food and nutrition information.


In [ ]:
# @title Setup Project and Vertex AI
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}

print(PROJECT_ID)

STAGING_BUCKET = "gs://dsf345gjt"

# Initialize the Vertex AI SDK
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)


In [ ]:
# @title Authenticate to BQ

import google.auth
from google.cloud import bigquery
creds, proj = google.auth.default()
print("ADC project:", proj)

# This just tests to ensure we are authenticated to BQ
bq = bigquery.Client(project=PROJECT_ID, credentials=creds)
bq.query("SELECT 1").result()
print("BigQuery query OK")

In [ ]:
# @title Set BigQuery Variables
DATASET_NAME = "usda_dataset"
DOWNLOAD_URL = "https://fdc.nal.usda.gov/fdc-datasets/FoodData_Central_foundation_food_csv_2025-04-24.zip"
EXTRACT_DIR = "fooddata_csv"

In [ ]:
# @title Download and Unzip Data File
import os
import requests
import zipfile
import shutil

# --- Config ---
url = DOWNLOAD_URL
zip_filename = "fooddata.zip"
extract_dir = EXTRACT_DIR  # expects EXTRACT_DIR to be defined in a previous cell

# --- Always start from scratch ---
if os.path.exists(zip_filename):
    try:
        os.remove(zip_filename)
        print(f"Removed existing zip: {zip_filename}")
    except Exception as e:
        print(f"Warning: could not remove existing zip ({zip_filename}): {e}")

if os.path.isdir(extract_dir):
    try:
        shutil.rmtree(extract_dir)
        print(f"Removed existing directory: {extract_dir}")
    except Exception as e:
        print(f"Warning: could not remove existing directory ({extract_dir}): {e}")

# --- Download fresh zip ---
print("Downloading dataset...")
with requests.get(url, stream=True, timeout=60) as response:
    response.raise_for_status()
    with open(zip_filename, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024 * 1024):  # 1 MB chunks
            if chunk:
                f.write(chunk)
print("Download complete.")

# --- Extract into a fresh directory ---
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_filename, "r") as zip_ref:
    zip_ref.extractall(extract_dir)
print("Extraction complete.")

# --- List all extracted files ---
print("Extracted files:")
for root, dirs, files in os.walk(extract_dir):
    for filename in sorted(files):
        print(os.path.join(root, filename))


In [ ]:
# @title Save CSV files to BigQuery
import os
import glob
from google.cloud import bigquery
from google.api_core.exceptions import NotFound, GoogleAPICallError, BadRequest

# Where you extracted the USDA CSVs (expects EXTRACT_DIR set earlier)
BASE_DIR = EXTRACT_DIR
MAX_BAD_RECORDS = 1000  # <-- allow up to 100 problematic rows per file

# Try to find the inner folder automatically (e.g., FoodData_Central_foundation_food_csv_2025-04-24)
subdirs = [d for d in glob.glob(os.path.join(BASE_DIR, "*")) if os.path.isdir(d)]
DATA_DIR = subdirs[0] if subdirs else BASE_DIR

print("Using data directory:", DATA_DIR)

client = bigquery.Client(project=PROJECT_ID)
dataset_ref = bigquery.DatasetReference(PROJECT_ID, DATASET_NAME)

# Ensure the dataset exists (create if it doesn't)
try:
    client.get_dataset(dataset_ref)
    print(f"Dataset {PROJECT_ID}.{DATASET_NAME} exists.")
except NotFound:
    ds = bigquery.Dataset(dataset_ref)
    ds.location = "US"  # change if needed
    client.create_dataset(ds)
    print(f"Created dataset {PROJECT_ID}.{DATASET_NAME} in location {ds.location}.")

# Load config for CSVs
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    autodetect=True,                         # Let BigQuery infer schema
    skip_leading_rows=1,                     # Header row
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    field_delimiter=",",
    quote_character='"',
    allow_quoted_newlines=True,
    encoding="UTF-8",
    max_bad_records=MAX_BAD_RECORDS,         # <-- tolerate up to N bad rows
    ignore_unknown_values=True,            # (Optional) tolerate extra columns
)

# Iterate over all CSV files and load each into a table named after the file (minus .csv)
csv_paths = sorted(glob.glob(os.path.join(DATA_DIR, "*.csv")))
if not csv_paths:
    raise FileNotFoundError(f"No CSV files found in {DATA_DIR}")

summary = []
failures = []

for csv_path in csv_paths:
    table_name = os.path.splitext(os.path.basename(csv_path))[0]
    table_ref = dataset_ref.table(table_name)
    print(f"\nLoading {csv_path} -> {PROJECT_ID}.{DATASET_NAME}.{table_name}")

    try:
        with open(csv_path, "rb") as f:
            load_job = client.load_table_from_file(f, table_ref, job_config=job_config)

        # Wait for completion; raises on job failure
        load_job.result()

        # Job may still have partial errors (within max_bad_records)
        if load_job.errors:
            print(f"Completed with {len(load_job.errors)} load errors (tolerated by max_bad_records={MAX_BAD_RECORDS}).")
            for err in load_job.errors[:5]:
                print("  -", err)

        table = client.get_table(table_ref)
        print(f"Loaded {table.num_rows} rows into {table.full_table_id}")
        summary.append((table_name, table.num_rows))

    except (BadRequest, GoogleAPICallError, Exception) as e:
        # Try to surface job-level error details if available
        err_details = None
        try:
            if 'load_job' in locals() and load_job is not None:
                if getattr(load_job, "error_result", None):
                    err_details = load_job.error_result
                elif getattr(load_job, "errors", None):
                    err_details = load_job.errors[:5]
        except Exception:
            pass

        print(f"ERROR loading {table_name}: {e}")
        if err_details:
            print("  Details:", err_details)
        failures.append((table_name, str(e)))

print("\n=== Load Summary ===")
for name, rows in summary:
    print(f"{name}: {rows} rows")

if failures:
    print("\n=== Failures ===")
    for name, msg in failures:
        print(f"{name}: {msg}")
else:
    print("\nNo failures 🎉")
